# Tennis players

In [153]:
# Import 
import pandas as pd 
import os 
import sys

sys.path.append(os.path.abspath(".."))

The raw data is available because I have already run the `download_atp_odds` and `download_atp` functions from `fetch_data.py`.

In [ ]:
# Read the raw data
atp_df = pd.read_csv("../data_storage/raw/atp_men_2000_2024/all_from_2000_to_2024.csv")
odds_df = pd.read_csv("../data_storage/raw/atp_odds_2001_2024/all_from_2001_to_2024.csv")

# We will merge the dataframes based on their date and the names of the two players
# Therefore, I will ensure that the relevant features have the same names
atp_df = atp_df.rename(columns={"tourney_date": "date", "winner_name": "winner", "loser_name": "loser"})
odds_df = odds_df.rename(columns={"Date": "date", "Winner": "winner", "Loser": "loser"})

# The odds data contains many features that we will not use in the future
odds_df = odds_df[["date", "winner", "loser", "Surface", "Wsets", "Lsets", "WRank", "LRank", "B365W", "B365L"]]
print(f"Number of rows:  ATP: {len(atp_df)} ,  Odds: {len(odds_df)}")

Number of rows:  ATP: 74906 ,  Odds: 63739


C:\Users\admin\AppData\Local\Temp\ipykernel_31388\3299882138.py:3: DtypeWarning: Columns (12,15,16,17,18,38) have mixed types. Specify dtype option on import or set low_memory=False.
  odds_df = pd.read_csv("../data/raw/atp_odds_2001_2024/all_from_2001_to_2024.csv")


## Merge ATP player data with their odds data


In [ ]:
from src.utils import merge_players_data
from importlib import reload
reload(merge_players_data)

df = merge_players_data.merge_on_name_and_date(atp_df, odds_df)
df.to_csv("../data_storage/raw/merged_data/merge_atp_odds.csv", index=False)
print("Save merged tennis players stats")
print(f"Size of merged dataframe {len(df)}")


Merge players data with there match odds: 100%|██████████| 63739/63739 [00:02<00:00, 22939.88it/s]


Save merged tennis players stats
Size of merged dataframe 12598


## Handle missing values
Logic which will manage the missing values are in the `src/utils/data_utils.py` file 

In [ ]:
from src.utils import data_utils
from importlib import reload
reload(data_utils)

# Drop the features if it's a rational reaches the threshold
df = data_utils.drop_hight_na_columns(df=df)
# Fill the missing values with there median if the ration is equal or lower then 3%
df = data_utils.fill_na_median(df)

# Print out the missing values
df.isna().sum()[df.isna().sum() > 0]

[INFO] Droping features Index(['winner_seed', 'winner_entry', 'loser_seed', 'loser_entry'], dtype='object')
[INFO] Columns which will be filled with there median: Index(['unnamed: 0', 'draw_size', 'match_num', 'winner_id', 'winner_ht',
       'winner_age', 'loser_id', 'loser_ht', 'loser_age', 'best_of', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stin', 'w_1stwon', 'w_2ndwon', 'w_svgms',
       'w_bpsaved', 'w_bpfaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stin',
       'l_1stwon', 'l_2ndwon', 'l_svgms', 'l_bpsaved', 'l_bpfaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'wsets', 'lsets', 'wrank'],
      dtype='object')


winner_key     356
loser_key      338
lrank           24
b365w         4464
b365l         4461
dtype: int64

## Convert feature data types
There are still missing values in the merged data, but that I will keep as it is so that I can have a closer look on them in the EDA file


In [ ]:
df.to_csv("../data_storage/interim/interim_data.csv", index=False)
print(f"Data save into the interim folder for feather checks. Number of rows: {len(df)}")

Data save into the interim folder for feather checks. Number of rows: 12598
